# <font color='blue'> DATA3650: Winter 2023
## <font color='darkcyan'> Assignment 4
 - Please download and use the input files provided with this notebook.
 - Write Python codes for each of the questions. Insert additional cells under each question when necessary.
 - You are not allowed to make any change in the input file.
 - You do **NOT** need to submit the input files. Please submit your answer in .ipynb file only.
 - Codes that shows error in execution will get 0.
 - Points: *22*

#### <font color='maroon'> Que 1: Load the 'credit_card_deafult' input file and store the dataset into a pandas DataFrame. 
- Create a pipeline to assemble several steps of data transfromation:
  - Step 1: Create a custom imputer to fill the missing values of 'SEX', 'EDUCATION' and 'MARRIAGE' columns by their corresponding most frequent values
  - Step 2: Create a custom imputer to fill the missing values for rest of the columns (except the target variable) by their corresponsing median values
  - Step 3: Apply standardization to transform the columns
     </font> 
- Display the DataFrame after applying pipeline. <br>
- Write comments in the cells to provide brief descriptions about your codes.

 **(Point: 10)**

In [134]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import SelectFromModel


In [135]:
df = pd.read_csv('credit_card_default.csv')
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [136]:
df.isnull().sum()

LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

In [137]:
# Define a custom imputer to fill missing values with the most frequent value for categorical columns
cat_imputer = SimpleImputer(strategy='most_frequent', missing_values=-1)
categorical_cols = ['SEX', 'EDUCATION', 'MARRIAGE']

# Define a custom imputer to fill missing values with the median for numerical columns
num_imputer = SimpleImputer(strategy='median')
numerical_cols = list(set(df.columns) - set(['ID', 'SEX', 'EDUCATION', 'MARRIAGE', 'default payment next month']))

# Create the pipeline
pipeline = Pipeline([
    ('cat_imputer', cat_imputer),
    ('num_imputer', num_imputer),
    ('scaler', StandardScaler())
])

# Apply the pipeline to the dataframe
df_transformed = pd.DataFrame(pipeline.fit_transform(df), columns=df.columns)

# Display the transformed dataframe
print(df_transformed.head())

   LIMIT_BAL       SEX  EDUCATION  MARRIAGE       AGE     PAY_0     PAY_2  \
0  -1.136720  0.810161   0.185828 -1.057295 -1.246020  1.788453  1.731713   
1  -0.365981  0.810161   0.185828  0.858557 -1.029047 -0.169171  1.731713   
2  -0.597202  0.810161   0.185828  0.858557 -0.161156 -0.169171 -0.060858   
3  -0.905498  0.810161   0.185828 -1.057295  0.164303 -0.169171 -0.060858   
4  -0.905498 -1.234323   0.185828 -1.057295  2.334029 -0.169171 -0.060858   

      PAY_3     PAY_4     PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  \
0 -0.028258  0.028099 -1.778427  ...  -0.672497  -0.663059  -0.652724   
1 -0.028258  0.028099  0.075614  ...  -0.621636  -0.606229  -0.597966   
2 -0.028258  0.028099  0.075614  ...  -0.449730  -0.417188  -0.391630   
3 -0.028258  0.028099  0.075614  ...  -0.232373  -0.186729  -0.156579   
4 -0.028258  0.028099  0.075614  ...  -0.346997  -0.348137  -0.331482   

   PAY_AMT1  PAY_AMT2  PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  \
0 -0.341942 -0.227086 -0.296801 -

#### <font color = 'maroon'> Que 2: Load the 'cancer' input file and store the dataset into a pandas DataFrame. Note that 'diagnosis' column is the traget variable.</font>     
- Fill missing values according to the data types and data distribution
- Perform data transformation
- Display null accuracy
- Apply a machine learning algorithm to perform better than null accuracy
- Select features using two feature selection methods: one of them of statistical based and another is model based.
- Apply the same machine learning algorithm on the selected features. Show accuracy. <br>
- Write comments in the cells to provide brief descriptions about your codes.
- Which feature selection technique you recommand for this dataset?

**(Point: 12)**

In [138]:
df2 = pd.read_csv('cancer.csv')
df2.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [139]:
# Set the 'diagnosis' column as the target variable
y = df2['diagnosis']

# Drop the 'id' and 'diagnosis' columns from the DataFrame
df2 = df2.drop(['id', 'diagnosis'], axis=1)
df2.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [140]:
# Check for missing values
print(df2.isnull().sum())

# Impute missing values with the median for numerical features
num_cols = df.select_dtypes(include=['float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())



radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64


In [141]:
# Transform categorical features into numerical features
X = pd.get_dummies(df2, drop_first=True)

In [142]:
# Calculate the null accuracy
null_accuracy = max(target.value_counts(normalize=True))
print('Null accuracy:', null_accuracy)

Null accuracy: 0.6274165202108963


In [143]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instantiate a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training set
rf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = rf.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9707602339181286


In [144]:
# Statistical based feature selection
selector = SelectKBest(f_classif, k=10)
selector.fit(X_train, y_train)
X_train_stat = selector.transform(X_train)
X_test_stat = selector.transform(X_test)

# Model based feature selection
selector = SelectFromModel(rf, prefit=True, threshold='median')
X_train_model = selector.transform(X_train)
X_test_model = selector.transform(X_test)

In [145]:
# Fit the Random Forest Classifier on the statistical based selected features
rf.fit(X_train_stat, y_train)

# Make predictions on the testing set
y_pred_stat = rf.predict(X_test_stat)

# Calculate the accuracy score
accuracy_stat = accuracy_score(y_test, y_pred_stat)
print('Accuracy with statistical based feature selection:', accuracy_stat)

# Fit the Random Forest Classifier on the model based selected features
rf.fit(X_train_model, y_train)

# Make predictions on the testing set
y_pred_model = rf.predict(X_test_model)

# Calculate the accuracy score
accuracy_model = accuracy_score(y_test, y_pred_model)
print('Accuracy with model based feature selection:', accuracy_model)

Accuracy with statistical based feature selection: 0.9473684210526315
Accuracy with model based feature selection: 0.9649122807017544


I would recommended that model-based feature selection as it provides the best accuracy compared to the statistical based feature selection model.